<a href="https://colab.research.google.com/github/reza610/psea/blob/main/Lisa_notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: get multiple files from github link

!wget -q -O test.zip "https://github.com/your_username/your_repo/archive/refs/heads/main.zip"
!unzip -qq test.zip
